In [2]:
import pandas as pd
from bokeh.charts import output_notebook, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.layouts import row
from bokeh.plotting import figure

In [16]:
dataset = pd.read_csv("data.csv",index_col=0,parse_dates=True)


rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset.workingday = dataset.workingday==1
dataset.holiday = dataset.holiday==1
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["season_s"] = dataset.season.map({1:"Printemps",2:"Ete",3:"Automne",4:"Hiver"})
dataset["weather_s"] = dataset.weather.map({1:"clear",2:"cloudy",3:"light rain",4:"crappy"})
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)

In [3]:
gb =dataset.groupby(dataset.index.weekday)["count"].mean()
b = Bar(gb, values="count")
output_notebook() 
#output_file("bar.html")
show(b)

Loading BokehJS ...

In [4]:
#dataset.hour = dataset.index.hour.map(dict( (i,"%i->%i" % (4*(i/4),4*(i/4)+3)) for i in range(24) ))


gg =  dataset.groupby(["weekday","hour"])["count"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="count",color="weekday")

show(b)

In [5]:
#dataset.hour = dataset.index.hour.map(dict( (i,"%i->%i" % (4*(i/4),4*(i/4)+3)) for i in range(24) ))

rich_weekday = lambda c: -1 if c["holiday"] else c["weekday"]

dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["rich_weekday"] = dataset.apply( rich_weekday, axis=1)
gg =  dataset.groupby(["weekday","hour"])["registered"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="registered",color="weekday")

show(b)

In [6]:
#dataset["casu_biker"] = dataset["count"] - dataset["registered"] 

gg =  dataset.groupby(["weekday","hour"])["casual"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="casual",color="weekday")
show(b)

In [7]:

gg =  dataset.groupby(["weekday","hour"])["registered"].mean().reset_index().rename(columns={"level_0":"weekday","level_1":"hour"})

gg["weekday"] = gg.weekday.map({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun",-1:"Holiday"})

b = Line(gg,x="hour",y="registered",color="weekday")
show(b)

In [8]:
check = dataset.registered +dataset.casual - dataset["count"]
check.value_counts()

0    10886
dtype: int64

In [9]:
h_ds = dataset.groupby( ["workingday","hour"])["count"].mean()

f = Bar(dataset,label="hour",values="count",agg="mean",group="holiday")
show(f)

In [10]:
f = Bar(dataset,label="season_s",values="count",agg="mean",group="holiday")
show(f)

In [11]:
f_casu = Bar(dataset,title = "casu as f(weather)",label="season_s",values="casual",agg="mean",group="weather_s")
f_reg = Bar(dataset,title = "registered as f(weather)",label="season_s",values="registered",agg="mean",group="weather_s")
show(row(f_casu,f_reg))

In [12]:
f = BoxPlot(dataset,title = "registered as f(weather)",label="season_s",values="count")
show(f)

In [13]:
#dataset[dataset.season==1]
dataset[dataset.weather==4]

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,weekday,season_s,weather_s,rich_weekday
datetime,,,,,,,,,,,,,,,,
2012-01-09 18:00:00,1,False,True,4,8.2,11.365,86,6.0032,6,158,164,18,0,Printemps,crappy,0


In [14]:
dataset["humidity_b"] = pd.cut(dataset.humidity,10)
f_casu = Bar(dataset,title="casu",label="humidity_b",values="casual",agg="mean",group="season_s")
f_reg = Bar(dataset,title="registered",label="humidity_b",values="registered",agg="mean",group="season_s")
show(row(f_casu,f_reg))

In [15]:
dataset["temp_b"] = pd.cut(dataset.temp,10)
f_casu = Bar(dataset,title="casu",label="temp_b",values="casual",agg="mean",group="season_s")
f_reg = Bar(dataset,title="registered",label="temp_b",values="registered",agg="mean",group="season_s")

show(row(f_casu,f_reg))

In [16]:
from bokeh.palettes import Spectral6
from bokeh.models import ColumnDataSource

grouped= dataset.groupby(["temp_b","humidity_b"])["count"].mean().reset_index()
grouped["temp"]=grouped.temp_b.str.extract("\((?P<temp>\d+\.\d+).*").apply(float)
grouped["humidity"]=grouped.humidity_b.str.extract("\((?P<temp>-?\d+(?:\.\d+)?).*")
grouped
#.apply(int)

mean_cnt =grouped["count"].mean()
grouped["sz"] = grouped["count"]/mean_cnt*15
colormap = dict( (i,Spectral6[i]) for i in range(len(Spectral6)))
cmap = pd.cut(grouped["count"],len(Spectral6),labels=Spectral6).tolist()
grouped["color_lgd"] = pd.cut(grouped["count"],len(Spectral6))

source = ColumnDataSource(dict(
    x=grouped.temp.tolist(),
    y=grouped.humidity.tolist(),
    color=cmap,
    sz=grouped["sz"].tolist(),
    label=grouped.color_lgd.tolist()
))
#.map(colormap)

#grouped
f = figure(plot_width=600, plot_height=600)
f.circle(x="x",y="y",size="sz", color="color",legend="label",source=source)
f.yaxis.axis_label = "Humidity"
f.xaxis.axis_label = "Temp (°C)"


show(f)

In [17]:
dataset["windspeed_g"] = pd.cut(dataset.windspeed,10)

b = Bar(dataset,label="windspeed_g",values="count",agg="mean",group="season_s")
show(b)

In [54]:
dset

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000


In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

dset =  pd.read_csv("data.csv",index_col=0,parse_dates=True)
label = dset["count"] 
del dset["count"], dset["registered"], dset["casual"]
dataset["hour"] =  dataset.index.hour
dataset["weekday"] = dataset.index.weekday
dataset["h_wd"] = dataset.weekday*100+dataset.hour

#dset_scaled = preprocessing.scale(dset)
pca = PCA(n_components="mle")

dt = DecisionTreeRegressor(max_depth=10)
clf = AdaBoostRegressor(n_estimators=1000,)
estimators = [('reduce_dim', pca), ('clf',clf)]
pipe = Pipeline(estimators)
scores = cross_val_score(clf, dset, label,cv=20,verbose=1,scoring="neg_mean_absolute_error",n_jobs=6)                          
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Avg dist to estimate: -125.65 (+/- 60.69)


[Parallel(n_jobs=6)]: Done   9 out of  20 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=6)]: Done  20 out of  20 | elapsed:    0.2s finished


In [ ]:
clf = DecisionTreeRegressor

In [40]:
from sklearn.model_selection import GridSearchCV
clf.get_params()
param_grid = [{'learning_rate':[1,10],'n_estimators':[10,100,500,1000,5000]}]
gscv = GridSearchCV(
    estimator = clf,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=5,
    verbose=2,
    n_jobs=6
)

f = gscv.fit(dset_scaled, label)

print f.best_estimator_
print f.best_score_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=10, learning_rate=1 ................................
[CV] n_estimators=10, learning_rate=1 ................................
[CV] n_estimators=10, learning_rate=1 ................................
[CV] n_estimators=10, learning_rate=1 ................................
[CV] n_estimators=10, learning_rate=1 ................................
[CV] n_estimators=100, learning_rate=1 ...............................
[CV] ................. n_estimators=10, learning_rate=1, total=   0.1s
[CV] n_estimators=100, learning_rate=1 ...............................
[CV] ................. n_estimators=10, learning_rate=1, total=   0.1s
[CV] ................. n_estimators=10, learning_rate=1, total=   0.1s
[CV] n_estimators=100, learning_rate=1 ...............................
[CV] n_estimators=100, learning_rate=1 ...............................
[CV] ................. n_estimators=10, learning_rate=1, total=   0.1s
[CV] ...........

[Parallel(n_jobs=6)]: Done  50 out of  50 | elapsed:   43.3s finished


AdaBoostRegressor(base_estimator=None, learning_rate=1, loss='linear',
         n_estimators=5000, random_state=None)
-139.447372432


In [26]:
from sklearn import svm
from sklearn import preprocessing
dset_scaled = preprocessing.scale(dset)
clf = svm.SVR(kernel='rbf',C=1,verbose=True, cache_size=1000)
#clf = svm.SVR(C=1e3,verbose=True)
scores = cross_val_score(clf, dset, label,cv=10,verbose=3,scoring="neg_median_absolute_error",n_jobs=6)
scores.mean()  
print("Avg dist to estimate: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[LibSVM][CV] .............................. , score=-112.005812, total=  14.9s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-87.822647, total=  14.9s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-89.552175, total=  15.0s
[CV]  ................................................................
[LibSVM][CV] ............................... , score=-96.561682, total=  15.2s
[CV]  .......................................

[Parallel(n_jobs=6)]: Done   3 out of  10 | elapsed:   15.1s remaining:   35.3s


[LibSVM][CV] .............................. , score=-105.270501, total=  15.6s
[LibSVM][CV] .............................. , score=-125.314444, total=  10.2s


[Parallel(n_jobs=6)]: Done   7 out of  10 | elapsed:   25.2s remaining:   10.8s


[LibSVM][CV] .............................. , score=-119.224953, total=  10.3s
[LibSVM][CV] .............................. , score=-128.349701, total=  10.5s
[LibSVM][CV] .............................. , score=-131.518268, total=  10.9s
Avg dist to estimate: -110.34 (+/- 29.93)


[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   25.9s finished


Accuracy: 0.03 (+/- 0.24)


In [6]:
cross_val_score?

In [18]:
dataset["count"].describe()

count    10886.000000
mean       191.574132
std        181.144454
min          1.000000
25%         42.000000
50%        145.000000
75%        284.000000
max        977.000000
Name: count, dtype: float64